# Pemodelan

In [76]:
# ! pip install pandas_datareader
# ! pip install ta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas_datareader as pdr
import datetime as dt
import ta

## Technical Indicators

In [81]:
class DataSaham():
    def __init__(self, code, start_date="2000-01-01", end_date=None, window=10, dropna=False):
        self.data = self.__scraping_saham(code, start_date, end_date)
        self.__get_technical_indicators(window)
        if dropna:
            self.data = self.data.dropna()
        
    def __scraping_saham(self, code, start_date="2000-01-01", end_date=None):
        start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
        end_date = dt.datetime.now() if end_date is None else dt.datetime.strptime(end_date, "%Y-%m-%d")
    
        data = pdr.get_data_yahoo(code, start_date, end_date)
        data.columns = data.columns.str.lower()
        return data.rename(columns={"adj close" : "adj_close"}).rename_axis('date')
    
    def __momentum(self, close, window=10):
        return close.rolling(window + 1).apply(lambda x : x[-1] - x[0])

    def __acc_dist_oscilator(self, high, low, close):
        return (high - close.shift(1)) / (high - low)
    
    def __get_technical_indicators(self, window=10):
        self.data['sma'] = ta.trend.sma_indicator(self.data.close, window=window)
        self.data['wma'] = ta.trend.wma_indicator(self.data.close, window=window)
        self.data['macd'] = ta.trend.macd(self.data.close)
        self.data['cci'] = ta.trend.cci(self.data.high, self.data.low, self.data.close)
        self.data['%k'] = ta.momentum.stoch(self.data.high, self.data.low, self.data.close)
        self.data['%d'] = ta.momentum.stoch_signal(self.data.high, self.data.low, self.data.close)
        self.data['rsi'] = ta.momentum.rsi(self.data.close)
        self.data['williams_r'] = ta.momentum.williams_r(self.data.high, self.data.low, self.data.close)
        self.data['mfi'] = ta.volume.money_flow_index(self.data.high, self.data.low, self.data.close, self.data.volume)
        self.data['momentum'] = self.__momentum(self.data.close, window=window)
        self.data['a/d'] = self.__acc_dist_oscilator(self.data.high, self.data.low, self.data.close)

In [82]:
bri = DataSaham('BBRI.JK', dropna=True)
bri.data

,high,low,open,close,volume,adj_close,sma,wma,macd,cci,%k,%d,rsi,williams_r,mfi,momentum,a/d
date,,,,,,,,,,,,,,,,,
2003-12-15,110.0,105.0,105.0,110.0,1.169155e+09,66.491806,104.75,105.545455,1.651442,162.202381,100.000000,91.666667,68.310546,-0.000000,73.007657,2.5,0.500000
2003-12-16,112.5,107.5,110.0,110.0,3.367050e+08,66.491806,105.25,106.500000,1.904385,209.302326,83.333333,94.444444,68.310546,-16.666667,74.382588,5.0,0.500000
2003-12-17,115.0,107.5,110.0,115.0,4.625900e+08,69.514160,106.50,108.272727,2.479718,262.271062,100.000000,94.444444,75.726371,-0.000000,76.093629,12.5,0.666667
2003-12-18,127.5,115.0,115.0,127.5,1.512910e+09,77.070053,108.75,112.090909,3.899370,373.080397,100.000000,94.444444,85.108585,-0.000000,80.712587,22.5,1.000000
2003-12-19,132.5,120.0,127.5,125.0,5.680800e+08,75.558868,111.00,115.045455,4.767765,282.464455,78.571429,92.857143,78.567800,-21.428571,80.531851,22.5,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-02,4120.0,4060.0,4120.0,4070.0,8.154630e+07,4070.000000,4113.00,4103.636364,-15.387185,-121.501272,29.166667,38.888889,42.417494,-70.833333,43.043584,-120.0,0.833333
2022-02-03,4180.0,4130.0,4180.0,4130.0,1.578988e+08,4130.000000,4113.00,4106.727273,-13.222065,3.681392,54.166667,37.500000,49.672645,-45.833333,45.314261,0.0,2.200000
2022-02-04,4210.0,4150.0,4170.0,4210.0,2.366029e+08,4210.000000,4121.00,4124.363636,-4.993300,88.624339,87.500000,56.944444,57.382817,-12.500000,53.588629,80.0,1.333333
